Here we will show an example of how to run deepblast on simulation data.

In [1]:
import os
from deepblast.sim import hmm_alignments
import argparse

We will first simulate multiple sequences from a single PFam family.
The resulting generated alignments will be used to train the model.

In [2]:
hmm = '../data/zf-C2H2.hmm'
n_alignments = 100
align_df = hmm_alignments(n=40, seed=0, n_alignments=n_alignments, hmmfile=hmm)

cols = [
    'chain1_name', 'chain2_name', 'tmscore1', 'tmscore2', 'rmsd',
    'chain1', 'chain2', 'alignment'
]
align_df.columns = cols

The simulated alignments will be split into training / testing and validation.

In [3]:
parts = n_alignments // 10
train_df = align_df.iloc[:parts * 8]
test_df = align_df.iloc[parts * 8:parts * 9]
valid_df = align_df.iloc[parts * 9:]

# save the files to disk.
train_df.to_csv('data/train.txt', sep='\t', index=None, header=None)
test_df.to_csv('data/test.txt', sep='\t', index=None, header=None)
valid_df.to_csv('data/valid.txt', sep='\t', index=None, header=None)

We will prepare the environment to make sure that the appropriate output directories exist to store the results.

In [4]:
from deepblast.trainer import LightningAligner
from pytorch_lightning import Trainer

output_dir = 'simulation_results'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [5]:
os.getcwd()

'/home/juermieboop/Documents/research/garfunkel/ipynb'

We will now create the arguments.  Below is the way to create this in a python environment.
This can also be recreated on a standard command line interface.

In [6]:
args = [
    '--train-pairs', f'{os.getcwd()}/data/train.txt',
    '--test-pairs', f'{os.getcwd()}/data/test.txt',
    '--valid-pairs', f'{os.getcwd()}/data/valid.txt',
    '--output-directory', output_dir,
    '--epochs', '10',
    '--batch-size', '1',    # this can only handle 1 batch atm. Need to change later.
    '--num-workers', '4',
    '--learning-rate', '1e-4',
    '--clip-ends', 'False',
    '--visualization-fraction', '0.5',
    '--gpus', '1'
]

We will then initialize the alignment model with the parameters we specified earlier.

In [7]:
parser = argparse.ArgumentParser(add_help=False)
parser = LightningAligner.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)
model = LightningAligner(args)

We can now train the model.

In [8]:
trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=1,
    # profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | aligner | NeedlemanWunschAligner | 23 M  


torch.Size([1, 27, 27]) torch.Size([1, 1])


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<numba.cuda.compiler.DeviceFunctionTemplate object at 0x7f1e0a6a60a0>) found for signature:
 
 >>> _forward_pass_device <CUDA device function>(array(float32, 2d, C), array(float32, 1d, C), array(float32, 3d, C))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function '_forward_pass_device <CUDA device function>': File: ../../../../../../Documents/research/garfunkel/deepblast/nw_cuda.py: Line 42.
    With argument(s): '(array(float32, 2d, C), array(float32, 1d, C), array(float32, 3d, C))':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   No implementation of function Function(<built-in function add>) found for signature:
    
    >>> add(array(float32, 1d, C), float32)
    
   There are 12 candidate implementations:
         - Of which 2 did not

The model diagnostics can be directly visualized in Tensorboard. Here, we show the losses, the accuracy and the alignment results.

In [ ]:
!ls lightning_logs

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs

In [ ]:
And we did this with just a few million parameters

In [ ]:
model.aligner